In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e7/sample_submission.csv
/kaggle/input/playground-series-s3e7/train.csv
/kaggle/input/playground-series-s3e7/test.csv


In [2]:
import numpy as np
import pandas as pd

In [3]:
!pip install autogluon.tabular[all]
!pip install ray==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.5/292.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.5/226.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2022.2.0
    Uninstalling dask-2022.2.0:
      Successfully uninstalled dask-2022.2.0
  Attempting uninstall: distributed
    Found existing installation: distributed 2022.2.0
    Uninstalling distributed-2022.2.0:
      Successfully uninstalled distributed-2022.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor

directory = '/kaggle/input/playground-series-s3e7/'

train = TabularDataset(directory + 'train.csv')
test = TabularDataset(directory + 'test.csv')

Loaded data from: /kaggle/input/playground-series-s3e7/train.csv | Columns = 19 / 19 | Rows = 42100 -> 42100
Loaded data from: /kaggle/input/playground-series-s3e7/test.csv | Columns = 18 / 18 | Rows = 28068 -> 28068


In [5]:
train.shape

(42100, 19)

In [6]:
label = 'booking_status'
time_limit = 3600 * 8
eval_metric = 'roc_auc'

predictor = TabularPredictor(label=label, eval_metric=eval_metric).fit(train, presets=['best_quality'], time_limit=time_limit)

submission = pd.read_csv(directory + 'sample_submission.csv')

submission[label] = predictor.predict(test)
submission.to_csv('submission.csv', index=False)
submission.head()

No path specified. Models will be saved in: "AutogluonModels/ag-20230221_090609/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 28800s
AutoGluon will save models to "AutogluonModels/ag-20230221_090609/"
AutoGluon Version:  0.6.2
Python Version:     3.7.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Feb 9 10:11:39 UTC 2023
Train Data Rows:    42100
Train Data Columns: 18
Label Column: booking_status
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Featu

,id,booking_status
0,42100,0
1,42101,0
2,42102,0
3,42103,0
4,42104,1


In [7]:
submission = pd.read_csv(directory + 'sample_submission.csv')

probablities = predictor.predict_proba(test, as_pandas=True)

submission[label] = probablities[1]
submission.to_csv('submission.csv', index=False)
submission.head()

,id,booking_status
0,42100,0.049919
1,42101,0.060700
2,42102,0.379514
3,42103,0.101670
4,42104,0.502816
